# Toronto Neighborhood Data

In [3]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df_raw = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df_raw = df_raw[~df_raw['PostalCode'].isnull()]  # to filter out bad rows

In [4]:
df_raw.head(10)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


### Delete all rows with Borough values as Not assigned

In [5]:
df_filtered = df_raw.drop(df_raw[df_raw.Borough == 'Not assigned'].index)

In [6]:
df_filtered.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Replace the Neighbourhood names with Borough names for Neighbourhood values showing Not assigned

In [7]:
df_filtered['Neighbourhood'].replace('Not assigned', df_filtered['Borough'], inplace=True)

In [8]:
df_filtered.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Group the rows with same Postal Code to show Neighbourhood in comma separated values

In [9]:
df_grouped = df_filtered.groupby('PostalCode').agg({'Borough':'first',
                                   'Neighbourhood': ','.join}).reset_index()

In [10]:
df_grouped.tail(10)

,PostalCode,Borough,Neighbourhood
93,M9A,Etobicoke,Islington Avenue
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar..."
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B..."
96,M9L,North York,Humber Summit
97,M9M,North York,"Emery,Humberlea"
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
102,M9W,Etobicoke,Northwest


### Print the shape of the dataframe

In [11]:
df_grouped.shape

(103, 3)

## Update Latitude and Longitude against each Postal Code/ Borough

In [12]:
import pandas as pd

In [13]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data Downloaded')

Data Downloaded


In [14]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
toronto_merged = df_grouped

In [16]:
toronto_merged = toronto_merged.join(df_coord.set_index('Postal Code'), on = 'PostalCode')

In [17]:
toronto_merged.tail(12)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
91,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout...",43.636258,-79.498509
92,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw...",43.628841,-79.520999
93,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
94,M9B,Etobicoke,"Cloverdale,Islington,Martin Grove,Princess Gar...",43.650943,-79.554724
95,M9C,Etobicoke,"Bloordale Gardens,Eringate,Markland Wood,Old B...",43.643515,-79.577201
96,M9L,North York,Humber Summit,43.756303,-79.565963
97,M9M,North York,"Emery,Humberlea",43.724766,-79.532242
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724


In [18]:
toronto_Neighbourhood = df_filtered
toronto_Neighbourhood = toronto_Neighbourhood.join(df_coord.set_index('Postal Code'), on = 'PostalCode')
toronto_Neighbourhood.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_Neighbourhood['Borough'].unique()),
        toronto_Neighbourhood.shape[0]
    )
)

The dataframe has 11 boroughs and 211 neighborhoods.


## Import necessary libraries for clustering

In [21]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [22]:
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


folium-0.5.0         | 45 KB     | #################################

In [23]:
toronto_Neighbourhood['Neighbourhood'].count()

211

In [24]:
toronto_nei_grouped = toronto_Neighbourhood.groupby('Borough').count()

In [25]:
toronto_nei_grouped.head()

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Central Toronto,17,17,17,17
Downtown Toronto,37,37,37,37
East Toronto,7,7,7,7
East York,6,6,6,6
Etobicoke,45,45,45,45


In [27]:
toronto_nei_grouped.sort_values(['Neighbourhood'], ascending=False)

,PostalCode,Neighbourhood,Latitude,Longitude
Borough,,,,
Etobicoke,45,45,45,45
North York,38,38,38,38
Downtown Toronto,37,37,37,37
Scarborough,37,37,37,37
Central Toronto,17,17,17,17
West Toronto,13,13,13,13
York,9,9,9,9
East Toronto,7,7,7,7
East York,6,6,6,6


In [28]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {},{}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963,-79.387207.


In [36]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(toronto_Neighbourhood['Latitude'],toronto_Neighbourhood['Longitude'],toronto_Neighbourhood['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup = label,
    color = 'red',
    fill_color ='blue',
    fill_opacity = 0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

In [37]:
CLIENT_ID = 'IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR' # your Foursquare ID
CLIENT_SECRET = 'G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF' # your Foursquare Secret
VERSION = '20190826' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR
CLIENT_SECRET:G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF


In [40]:
toronto_Neighbourhood.reset_index()
toronto_Neighbourhood.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.753259,-79.329656
4,M4A,North York,Victoria Village,43.725882,-79.315572
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
7,M6A,North York,Lawrence Heights,43.718518,-79.464763


### Explore the first Neighbourhood in the dataframe

In [41]:
toronto_Neighbourhood.loc[3,'Neighbourhood']

'Parkwoods'

In [43]:
neighbourhood_latitude = toronto_Neighbourhood.loc[3,'Latitude']
neighbourhood_longitude = toronto_Neighbourhood.loc[3,'Longitude']
neighbourhood_name = toronto_Neighbourhood.loc[3,'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [44]:
LIMIT = 100
RADIUS = 500
latitude = neighbourhood_latitude
longitude= neighbourhood_longitude

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, RADIUS, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=IKQA4XDS5FMGR3ADWAEY055AU5OX1XF1DHOLJQXF1GB5KGPR&client_secret=G2IU3NDY4QFW2OE2VLHAMXOCIPNPVJKPT2U31QAVCNCKMUWF&v=20190826&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d66495a48b1e100378953b1'},
 'response': {'venues': [{'id': '4e8d9dcdd5fbbbb6b3003c7b',
    'name': 'Brookbanks Park',
    'location': {'address': 'Toronto',
     'lat': 43.751976046055574,
     'lng': -79.33214044722958,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.751976046055574,
       'lng': -79.33214044722958}],
     'distance': 245,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566984538',
    'hasPerk': False},
   {'id': '4e039defd22d4cebf370894a',
    'name': 'Cassandra Public School',
    'location': {'address': '45 Cassandra Blvd',


In [46]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [48]:
import json
from pandas.io.json import json_normalize

In [50]:
venues = results['response']['venues']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['categories', 'hasPerk', 'id', 'location.address', 'location.cc',
       'location.city', 'location.country', 'location.crossStreet',
       'location.distance', 'location.formattedAddress',
       'location.labeledLatLngs', 'location.lat', 'location.lng',
       'location.neighborhood', 'location.postalCode', 'location.state',
       'name', 'referralId', 'venuePage.id'],
      dtype='object')

In [52]:
filtered_columns = ['name','categories','location.lat','location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Cassandra Public School,School,43.748291,-79.328889
2,Pheasant Run Golf Course,Golf Course,43.758386,-79.337191
3,Joey,Burger Joint,43.753441,-79.321640
4,CAPREIT Toronto Townhomes - 56 Cassandra Blvd,Building,43.753920,-79.322400


In [53]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Explore Neighbourhoods in Toronto

<a id='item2'></a>

In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [55]:
toronto_venues = getNearbyVenues(names=toronto_Neighbourhood['Neighbourhood'],
                                   latitudes=toronto_Neighbourhood['Latitude'],
                                   longitudes=toronto_Neighbourhood['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Regent Park
Lawrence Heights
Lawrence Manor
Queen's Park
Islington Avenue
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
CFB Toronto
Downsview East
The Danforth West
Riverdale
Design E

In [56]:
print(toronto_venues.shape)
toronto_venues.head()

(4476, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


In [57]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
Agincourt North,3,3,3,3,3,3
Albion Gardens,10,10,10,10,10,10
Alderwood,8,8,8,8,8,8
Bathurst Manor,18,18,18,18,18,18
Bathurst Quay,16,16,16,16,16,16
Bayview Village,4,4,4,4,4,4
Beaumond Heights,10,10,10,10,10,10


In [58]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 276 uniques categories.


<a id='item3'></a>

### Analyze each Neighbourhood

In [60]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [61]:
toronto_onehot.shape

(4476, 277)

In [62]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.030000,...,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,Agincourt North,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,Albion Gardens,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Alderwood,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Bathurst Manor,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,Bathurst Quay,0.0,0.000000,0.062500,0.0625,0.0625,0.125,0.125,0.125,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Bayview Village,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Beaumond Heights,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.000000,...,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,Bedford Park,0.0,0.000000,0.000000,0.0000,0.0000,0.000,0.000,0.000,0.045455,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [63]:
toronto_grouped.shape

(203, 277)

In [67]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.03


----Agincourt----
            venue  freq
0  Breakfast Spot  0.25
1          Lounge  0.25
2  Clothing Store  0.25
3    Skating Rink  0.25
4   Metro Station  0.00


----Agincourt North----
                        venue  freq
0                  Playground  0.33
1            Asian Restaurant  0.33
2                        Park  0.33
3  Modern European Restaurant  0.00
4               Moving Target  0.00


----Albion Gardens----
                 venue  freq
0        Grocery Store   0.2
1          Pizza Place   0.1
2          Video Store   0.1
3  Fried Chicken Joint   0.1
4          Coffee Shop   0.1


----Alderwood----
            venue  freq
0     Pizza Place  0.25
1             Pub  0.12
2    Skating Rink  0.12
3  Sandwich Place  0.12
4     Coffee Shop  0.12


----Bathurst Manor----
              venue  freq
0       Coffe

In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [96]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant,Cosmetics Shop,Hotel,Burger Joint,Restaurant,American Restaurant
1,Agincourt,Breakfast Spot,Lounge,Skating Rink,Clothing Store,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
2,Agincourt North,Playground,Park,Asian Restaurant,Yoga Studio,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,Albion Gardens,Grocery Store,Pizza Place,Coffee Shop,Sandwich Place,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pharmacy,Video Store,Greek Restaurant
4,Alderwood,Pizza Place,Skating Rink,Coffee Shop,Pub,Sandwich Place,Pharmacy,Gym,Greek Restaurant,Deli / Bodega,Eastern European Restaurant


In [94]:
neighborhoods_venues_sorted.shape

(203, 12)

In [97]:
toronto_Neighbourhood.shape

(211, 5)

### Cluster Neighbourhoods

In [98]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=4).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [99]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels'] = neighborhoods_venues_sorted['Cluster Labels'].astype(int)

toronto_merged = toronto_Neighbourhood

# merge toronto_grouped with toronto_Neighbourhood to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how='inner')

toronto_merged.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M3A,North York,Parkwoods,43.753259,-79.329656,0,Bus Stop,Park,Food & Drink Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,M4A,North York,Victoria Village,43.725882,-79.315572,2,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
5,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater
6,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636,2,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater
7,M6A,North York,Lawrence Heights,43.718518,-79.464763,2,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Gift Shop,Clothing Store,Boutique,Vietnamese Restaurant,Cuban Restaurant


In [100]:
toronto_merged['Cluster Labels'].unique().tolist()

[0, 2, 4, 1, 3]

In [101]:
toronto_merged.shape

(205, 16)

In [102]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,2,Pizza Place,Hockey Arena,Coffee Shop,Portuguese Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant
5,Downtown Toronto,2,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater
6,Downtown Toronto,2,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Gym / Fitness Center,Mexican Restaurant,Breakfast Spot,Theater
7,North York,2,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Gift Shop,Clothing Store,Boutique,Vietnamese Restaurant,Cuban Restaurant
8,North York,2,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Arts & Crafts Store,Gift Shop,Clothing Store,Boutique,Vietnamese Restaurant,Cuban Restaurant
9,Queen's Park,2,Coffee Shop,Gym,Diner,Park,College Auditorium,Smoothie Shop,Seafood Restaurant,Burger Joint,Burrito Place,Sandwich Place
15,North York,2,Japanese Restaurant,Gym / Fitness Center,Baseball Field,Café,Basketball Court,Caribbean Restaurant,Drugstore,Dog Run,Doner Restaurant,Donut Shop
16,East York,2,Pizza Place,Fast Food Restaurant,Pharmacy,Gastropub,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center,Discount Store
17,East York,2,Pizza Place,Fast Food Restaurant,Pharmacy,Gastropub,Bank,Athletics & Sports,Intersection,Breakfast Spot,Gym / Fitness Center,Discount Store
18,Downtown Toronto,2,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Café,Italian Restaurant,Pizza Place,Plaza,Lingerie Store,Tea Room
